# Summary
This notebook is used to evaluate CTPN text detection, which can be built from here:
<br>
https://github.com/eragonruan/text-detection-ctpn

# Imports

In [1]:
import os

In [2]:
import numpy as np

In [3]:
# To parse coco annot
import coco_text
# To eval prediction json 
import coco_evaluation

# Constants

In [4]:
# Cocotext annotation data, can be downloaded here: https://s3.amazonaws.com/cocotext/COCO_Text.zip
GROUND_TRUTH_PATH = r'/home/rudi/Documents/Data/cocotext/COCO_Text.json'
# Bounding box annotations generated by CTPN, each record containing: 
# image id
# bounding box [top left x, top left y, width, height]
# confidence score
EVAL_PATH = r'/home/rudi/Documents/Data/cocotext/cocotext_ctpn_output_score.json'
# To compare the bounding box if needed
IMAGES_DIR = r'/home/rudi/Documents/Data/cocotext/train2014_text_valid'

# Initialization

In [5]:
# Ground truth annotations
gt = coco_text.COCO_Text(GROUND_TRUTH_PATH)

loading annotations into memory...
0:00:01.944672
creating index...
index created!


In [6]:
gt.info()

url: http://vision.cornell.edu/se3/coco-text/
date_created: 2017-03-28
version: 1.4
description: This is 1.4 version of the 2017 COCO-Text dataset.
author: COCO-Text group


In [7]:
# Evaluated predictions
ev = gt.loadRes(EVAL_PATH)

Loading and preparing results...     
DONE (t=15.14s)


In [8]:
# Get the list of annotation and image ids with certain filter
# Default used for comparison: legible text, machine printed (screw handwriting)
valid_image_ids = gt.getImgIds(imgIds=gt.train, 
                    catIds=[('legibility','legible'),('class','machine printed')])
valid_annotation_ids = gt.getAnnIds(imgIds=gt.train, 
                        catIds=[('legibility','legible'),('class','machine printed')])

In [9]:
# Build a dictionary of image IDs to their filepath
# For visualisation and other misc stuff
image_paths = {}
for image_name in os.listdir(IMAGES_DIR):
    image_id = int(image_name.split('.')[0].split('_')[-1])
    image_paths[image_id] = os.path.join(IMAGES_DIR,image_name)

# Main

In [19]:
# Sample annotation comparison
image_id = 81

gt_ann = gt.loadAnns(gt.getAnnIds(imgIds=image_id))
eval_ann = ev.loadAnns(ev.getAnnIds(imgIds=image_id))

print('Ground truth box of image '+str(image_id))
print(gt_ann[0]['bbox'])
print('\nEvaluated box of image '+str(image_id))
print(eval_ann[0]['bbox'])
print('\nIOU: '+str(coco_evaluation.iou_score(gt_ann[0]['bbox'], eval_ann[0]['bbox'])))

Ground truth box of image 81
[101, 173, 127.8800759193141, 18.483669688596507]

Evaluated box of image 81
[90, 175, 147, 19]

IOU: 0.6914066132540398


In [14]:
from importlib import reload 
reload(coco_evaluation)

<module 'coco_evaluation' from '/home/rudi/Documents/Notebook/Data_Anal/coco_text/coco_evaluation.py'>

In [15]:
detection_results = coco_evaluation.getDetections(gt,ev,
                                                  imgIds=valid_image_ids,
                                                  annIds=valid_annotation_ids,
                                                  detection_threshold=0.5)

In [16]:
print(f"True positive count : {len(detection_results['true_positives'])}")
print(f"False positive count : {len(detection_results['false_positives'])}")
print(f"False negative count : {len(detection_results['false_negatives'])}")

True positive count : 12656
False positive count : 54123
False negative count : 53064
